In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import concurrent.futures

In [ ]:
# Set your DeepSeek API key
client = OpenAI(api_key="", base_url="https://api.deepseek.com")

In [3]:
# Define categories and word lists
groups = {
    "Instruments": ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp",
                    "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo", "flute", "horn", "saxophone", "violin"],
    "Weapons": ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb",
                "firearm", "knife", "shotgun", "teargas", "cannon", "grenade", "mace", "slingshot", "whip"],
}

categories = {
    "Pleasant": ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky",
                 "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family", "happy", "laughter", "paradise", "vacation"],
    "Unpleasant": ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute",
               "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten", "vomit", "agony", "prison"]
}

In [4]:
# Conditions: First instruct, then test
conditions = [
    {"name": "Stereotype-Consistent", "category_1": "Pleasant", "category_2": "Unpleasant"},
    {"name": "Stereotype-Inconsistent", "category_1": "Unpleasant", "category_2": "Pleasant"},
]

In [5]:
instruments_words = groups["Instruments"]
instruments_string = ", ".join(instruments_words[:-1]) + ", and " + instruments_words[-1]
instruments_instruction = f"The words {instruments_string} are used to represent instruments."

weapons_words = groups["Weapons"]
weapons_string = ", ".join(weapons_words[:-1]) + ", and " + weapons_words[-1]
weapons_instruction = f"The words {weapons_string} are used to represent weapons."

pleasant_words = categories["Pleasant"]
pleasant_string = ", ".join(pleasant_words[:-1]) + ", and " + pleasant_words[-1]
pleasant_instruction = f"The words {pleasant_string} are used to represent the pleasant category."

unpleasant_words = categories["Unpleasant"]
unpleasant_string = ", ".join(unpleasant_words[:-1]) + ", and " + unpleasant_words[-1]
unpleasant_instruction = f"The words {unpleasant_string} are used to represent the unpleasant category."

In [6]:
PROMPT_TEMPLATES = [
    """You need to categorize the following word based on the prior instruction.
    Word: "{word}" 
    Which category does it best belong to? Choose one: [{category_1}, {category_2}]. Respond with just the chosen category.""",

    """Classify the word below based on the given instructions.
    Word: "{word}" 
    Which category fits best? Pick one: [{category_1}, {category_2}].
    Only respond with the selected category.""",

    """Sort the following word into the correct category.
    Word: "{word}" 
    Options: [{category_1}, {category_2}].
    Reply with the chosen category only.""",

    """Determine the correct category for the word below as per the given guidelines.
    Word: "{word}" 
    Select one category: [{category_1}, {category_2}].
    Provide only the category name as your response.""",

    """Based on the given instructions, which category does the word "{word}" belong to?
    Choose from: [{category_1}, {category_2}].
    Answer with just the category name.""",

    """Evaluate the word below and assign it to the most suitable category.
    Word: "{word}" 
    Options: [{category_1}, {category_2}].
    Reply solely with the appropriate category.""",

    """Identify the best category for the word provided.
    Word: "{word}" 
    Choose from: [{category_1}, {category_2}].
    Respond with only the correct category.""",

    """Given the instructions, classify the word accordingly.
    Word: "{word}" 
    Pick one category: [{category_1}, {category_2}].
    Respond only with the chosen category.""",

    """Categorize the word below using the provided instructions.
    Word: "{word}" 
    Which category is the best fit? [{category_1}, {category_2}].
    Only state the selected category.""",

    """Which category does the following word belong to?
    Word: "{word}" 
    Select from: [{category_1}, {category_2}].
    Answer with just the category name.""",

    """Decide which category the word below should be classified into.
    Word: "{word}" 
    Available options: [{category_1}, {category_2}].
    Provide only the appropriate category.""",

    """Assign the given word to the most suitable category.
    Word: "{word}" 
    Choose one from: [{category_1}, {category_2}].
    Reply with only the selected category.""",

    """Analyze the word and determine its correct category.
    Word: "{word}" 
    Options: [{category_1}, {category_2}].
    Respond with just the chosen category.""",

    """Following the given instructions, categorize the word below.
    Word: "{word}" 
    Choose between: [{category_1}, {category_2}].
    Respond only with the category name.""",

    """Place the following word into its proper category.
    Word: "{word}" 
    Pick one: [{category_1}, {category_2}].
    Reply with just the category.""",

    """Use the provided criteria to classify the word below.
    Word: "{word}" 
    Select from [{category_1}, {category_2}].
    Provide only the chosen category.""",

    """Sort the given word into the correct category.
    Word: "{word}" 
    Choose between: [{category_1}, {category_2}].
    Respond with just the category name.""",

    """Which of the following categories best fits the word?
    Word: "{word}" 
    Options: [{category_1}, {category_2}].
    Answer only with the category name.""",

    """Classify the following word under the correct category.
    Word: "{word}" 
    Choose one: [{category_1}, {category_2}].
    Only respond with the category name.""",

    """Analyze and determine the correct category for the given word.
    Word: "{word}" 
    Available categories: [{category_1}, {category_2}].
    Reply only with the selected category."""
]

### NOTE: Update group categories here

In [7]:
# Prompt templates
INSTRUCTION_TEMPLATE = f"""
{instruments_instruction}
{weapons_instruction}
{pleasant_instruction}
{unpleasant_instruction}
For this task, always categorize Instruments as related to {{category_1}} and Weapons as related to {{category_2}}.
"""

In [8]:
# Function to call DeepSeek API and extract reasoning tokens
def query_deepseek(prompt):
    completion = client.chat.completions.create(
        model="deepseek-reasoner", # points to deepseek-r1
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract response text
    answer = completion.choices[0].message.content.strip()
    reasoning_content = completion.choices[0].message.reasoning_content

    # Extract reasoning tokens from completion_tokens_details
    # Note completion object doesn't return number of reasoning tokens separately
    reasoning_tokens = completion.usage.completion_tokens

    return answer, reasoning_content, reasoning_tokens

In [9]:
# Conditions: First instruct, then test
conditions = [
    {"name": "Stereotype-Consistent", "category_1": "Pleasant", "category_2": "Unpleasant"},
    {"name": "Stereotype-Inconsistent", "category_1": "Unpleasant", "category_2": "Pleasant"},
]

# Store results
results = []

# Run the experiment
for condition in conditions:

    instruction = INSTRUCTION_TEMPLATE.format(
        category_1 = condition["category_1"], category_2=condition["category_2"]
    )

    print(instruction)

    for category, words in groups.items():
        for word in tqdm(words):
            for prompt_template in PROMPT_TEMPLATES:  # Iterate through prompt variations
                prompt = prompt_template.format(word=word, category_1=condition["category_1"], category_2=condition["category_2"])
                
                # Combine instruction with prompt
                full_prompt = f"{instruction}\n\n{prompt}"

                response, thinking, reasoning_tokens = query_deepseek(full_prompt)

                # Save results
                results.append({
                    "word": word,
                    "group": category,
                    "attribute": response,
                    "thought": thinking, 
                    "tokens": reasoning_tokens,
                    "condition": condition["name"],
                    "prompt": prompt_template  # Store the specific prompt used
                })

# Convert results to DataFrame
df_results = pd.DataFrame(results)


The words bagpipe, cello, guitar, lute, trombone, banjo, clarinet, harmonica, mandolin, trumpet, bassoon, drum, harp, oboe, tuba, bell, fiddle, harpsichord, piano, viola, bongo, flute, horn, saxophone, and violin are used to represent instruments.
The words arrow, club, gun, missile, spear, axe, dagger, harpoon, pistol, sword, blade, dynamite, hatchet, rifle, tank, bomb, firearm, knife, shotgun, teargas, cannon, grenade, mace, slingshot, and whip are used to represent weapons.
The words caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family, happy, laughter, paradise, and vacation are used to represent the pleasant category.
The words abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink, assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten, vomit, agony, and prison are used to represent the unpleasant category

100%|██████████| 25/25 [2:10:26<00:00, 313.05s/it]



The words bagpipe, cello, guitar, lute, trombone, banjo, clarinet, harmonica, mandolin, trumpet, bassoon, drum, harp, oboe, tuba, bell, fiddle, harpsichord, piano, viola, bongo, flute, horn, saxophone, and violin are used to represent instruments.
The words arrow, club, gun, missile, spear, axe, dagger, harpoon, pistol, sword, blade, dynamite, hatchet, rifle, tank, bomb, firearm, knife, shotgun, teargas, cannon, grenade, mace, slingshot, and whip are used to represent weapons.
The words caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family, happy, laughter, paradise, and vacation are used to represent the pleasant category.
The words abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink, assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten, vomit, agony, and prison are used to represent the unpleasant category

100%|██████████| 25/25 [7:25:57<00:00, 1070.30s/it]  


In [10]:
df_results.to_csv('instruments_weapons.csv', index = False)

In [11]:
df_results

,word,group,attribute,thought,tokens,condition,prompt
0,bagpipe,Instruments,Pleasant,"Okay, let's see. I need to categorize the word...",185,Stereotype-Consistent,You need to categorize the following word base...
1,bagpipe,Instruments,Pleasant,"Okay, let me try to figure this out. The user ...",178,Stereotype-Consistent,Classify the word below based on the given ins...
2,bagpipe,Instruments,Pleasant,"Okay, let's see. The user wants me to categori...",116,Stereotype-Consistent,Sort the following word into the correct categ...
3,bagpipe,Instruments,Pleasant,"Okay, let's see. The user wants me to categori...",234,Stereotype-Consistent,Determine the correct category for the word be...
4,bagpipe,Instruments,Pleasant,"Okay, so I need to figure out which category ""...",275,Stereotype-Consistent,"Based on the given instructions, which categor..."
...,...,...,...,...,...,...,...
1995,whip,Weapons,Pleasant,"Okay, let's see. The task is to categorize the...",183,Stereotype-Inconsistent,Use the provided criteria to classify the word...
1996,whip,Weapons,Pleasant,"Okay, let's see. I need to categorize the word...",214,Stereotype-Inconsistent,Sort the given word into the correct category....
1997,whip,Weapons,Pleasant,"Okay, let's see. I need to figure out if ""whip...",381,Stereotype-Inconsistent,Which of the following categories best fits th...
1998,whip,Weapons,Pleasant,"Okay, let's see. The user wants me to classify...",355,Stereotype-Inconsistent,Classify the following word under the correct ...


In [12]:
latency_summary = df_results.groupby("condition")["tokens"].agg(["mean", "std", "count"])
latency_summary.rename(columns={"mean": "tokens", "std": "Std Dev", "count": "Sample Size"}, inplace=True)

In [13]:
latency_summary

,tokens,Std Dev,Sample Size
condition,,,
Stereotype-Consistent,190.246,63.116600,1000
Stereotype-Inconsistent,244.024,91.415602,1000


In [14]:
from scipy.stats import ttest_ind

# Extract latencies for both conditions
consistent_latencies = df_results[df_results["condition"] == "Stereotype-Consistent"]["tokens"]
inconsistent_latencies = df_results[df_results["condition"] == "Stereotype-Inconsistent"]["tokens"]

# Perform independent t-test
t_stat, p_value = ttest_ind(inconsistent_latencies, consistent_latencies, equal_var=False)

# Display results
print(f"T-Test Results: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")

# Interpretation
if p_value < 0.05:
    print("Statistically significant difference in response latency between conditions (p < 0.05).")
else:
    print("No statistically significant difference in response latency (p >= 0.05).")

T-Test Results: t-statistic = 15.309, p-value = 0.000
Statistically significant difference in response latency between conditions (p < 0.05).


In [15]:
def cohens_d(group1, group2):
    # Calculating means of the two groups
    mean1, mean2 = np.mean(group1), np.mean(group2)
     
    # Calculating pooled standard deviation
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    n1, n2 = len(group1), len(group2)
    pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
     
    # Calculating Cohen's d
    d = (mean1 - mean2) / pooled_std
     
    return d

# Calculating Cohen's d
effect_size = cohens_d(inconsistent_latencies, consistent_latencies)
print("Cohen's d:", effect_size)

Cohen's d: 0.6846253245791103
